### Imports

In [36]:
import pandas as pd
import requests
import statistics

### Parse JSON

In [37]:
# File path

PATH = 'C:\\Users\\Owner\\Documents\\Big Ideas Lab\\'

In [38]:
# load data

import json

with open(PATH+'restaurant_data.json') as json_file:
    data = json.load(json_file)
    numRest = len(data['Restaurants']) # number of restaurants in file
    restList = range(0,numRest)

### Function to call USDA for single item nutrition information (Currently -> cal, sugar, fat, carbs)

In [60]:
#searching the USDA database based on an individual ingredient.
#this function is used below. 

def nutri_search(menu_item):

    #Set search parameters -- find the closest USDA entry that is similar to our ingredient. 
    
    parameters1 = {
        'api_key' : 'ZLuguGk9HC149FasGPnpE8ldNrf8tZOevBiI7gdD',
        'q' : '{}'.format(menu_item)
    }
    
    # API call to file the ndbno (an index number that the USDA uses for nutrition information)

    response = requests.get('https://api.nal.usda.gov/ndb/search/?format=json', params = parameters1)
    JSON = response.json()
    ndbno = JSON['list']['item'][0]['ndbno']

    #Second set of parameters to call for nutrition information with our new ndbno
    
    parameters2 = {
        'api_key' : 'ZLuguGk9HC149FasGPnpE8ldNrf8tZOevBiI7gdD',
        'nutrients':["208","269","298","606","605","205","203","291","307","324","304","431","417","306"], # nutrient_nbr in nutrient.csv
        'ndbno' : '{}'.format(ndbno)
    }

    # Second API call to get nutrition information. In order, we're getting calories, sugars, fats, and carbs
    # !!! Let me know if you want other parameters. Options can be found in the USDA download (not the API) under
    # nutrient.csv file
    
    response = requests.get('https://api.nal.usda.gov/ndb/nutrients/?format=json', params = parameters2)
    JSON = response.json()
    
    # cals
    cal = JSON['report']['foods'][0]['nutrients'][0]['value']
    if cal == '--':
        cal = 0
    else: 
        cal=float(cal)
        
    # macronutrients
    sugars = JSON['report']['foods'][0]['nutrients'][1]['value']
    if sugars == '--':
        sugars = 0
    else : 
        sugars=float(sugars)
    totfat = JSON['report']['foods'][0]['nutrients'][2]['value']
    if totfat == '--':
        totfat = 0
    else : 
        totfat=float(totfat)
    satfat = JSON['report']['foods'][0]['nutrients'][3]['value']
    if satfat == '--':
        satfat = 0
    else : 
        satfat=float(satfat)
    transfat = JSON['report']['foods'][0]['nutrients'][4]['value']
    if transfat == '--':
        transfat = 0
    else : 
        transfat=float(transfat)
    carbs = JSON['report']['foods'][0]['nutrients'][5]['value']
    if carbs == '--':
        carbs = 0
    else : 
        carbs=float(carbs)
    protein = JSON['report']['foods'][0]['nutrients'][6]['value']
    if protein == '--':
        protein = 0
    else : 
        protein=float(protein)
    fiber = JSON['report']['foods'][0]['nutrients'][7]['value']
    if fiber == '--':
        fiber = 0
    else : 
        fiber=float(fiber)
    
    # minerals
    sod = JSON['report']['foods'][0]['nutrients'][8]['value']
    if sod == '--':
        sod = 0
    else : 
        sod=float(sod)
    mag = JSON['report']['foods'][0]['nutrients'][9]['value']
    if mag == '--':
        mag = 0
    else : 
        mag=float(mag)
    folic = JSON['report']['foods'][0]['nutrients'][10]['value']
    if folic == '--':
        folic = 0
    folate = JSON['report']['foods'][0]['nutrients'][11]['value']
    if folate == '--':
        folate = 0
    folate = float(folic) + float(folate)
    potass = JSON['report']['foods'][0]['nutrients'][12]['value']
    if potass == '--':
        potass = 0
    else : 
        potass=float(potass)
    
    # vitamins
    vd = JSON['report']['foods'][0]['nutrients'][13]['value']
    if vd == '--':
        vd = 0
    else : 
        vd=float(vd)
    
    return cal, sugars, totfat, satfat, transfat, carbs, protein, fiber, sod, mag, folate, potass, vd

### Search and return all estimated cal, sugars, fats, and carbs for each menu item given retaurant name. 

In [66]:
def create_list_rest_dictionaries(data):
    restaurants = data['Restaurants']
    rest_dict_list = []
    for restaurant in restaurants:
        rest_dict_list.append(dict(restaurant))  
    for i in range(0,len(rest_dict_list)):
        for menu_item in list(rest_dict_list[i]['Menu'].keys()):
            ingredients = rest_dict_list[i]['Menu'][menu_item]['ingredients']
            rest_dict_list[i]['Menu'][menu_item]['nutrition'] = {}        
            for ingredient in ingredients:
                rest_dict_list[i]['Menu'][menu_item]['nutrition'][ingredient] = {}
                rest_dict_list[i]['Menu'][menu_item]['nutrition'][ingredient] = nutri_search(ingredient)
    return rest_dict_list
rest_dict_list = create_list_rest_dictionaries(data)
print(rest_dict_list)

KeyError: 'report'

### Nutrition information from return above -> Score it. (Sabrina) 

In [9]:
# Dictionaries for recommended nutrient amounts by gender, age, activity

recommended_nutrient_dict_M_S = {'cal':2400/3,'sugars':120/3,'totfat':95/3,'satfat':30/3,'transfat':2/3,'carbs':130/3,'protein':56/3,'fiber':38/3,'sod':2300/3,'mag':420/3,'totfolate':400/3,'potass':4700/3,'vd':600/3}
recommended_nutrient_dict_M_M = {'cal':2600/3,'sugars':120/3,'totfat':95/3,'satfat':30/3,'transfat':2/3,'carbs':130/3,'protein':56/3,'fiber':38/3,'sod':2300/3,'mag':420/3,'totfolate':400/3,'potass':4700/3,'vd':600/3}
recommended_nutrient_dict_M_A = {'cal':2800/3,'sugars':120/3,'totfat':95/3,'satfat':30/3,'transfat':2/3,'carbs':130/3,'protein':56/3,'fiber':38/3,'sod':2300/3,'mag':420/3,'totfolate':400/3,'potass':4700/3,'vd':600/3}
recommended_nutrient_dict_F_S = {'cal':1800/3,'sugars':90/3,'totfat':70/3,'satfat':20/3,'transfat':2/3,'carbs':130/3,'protein':46/3,'fiber':25/3,'sod':2300/3,'mag':320/3,'totfolate':400/3,'potass':4700/3,'vd':600/3}
recommended_nutrient_dict_F_M = {'cal':2000/3,'sugars':90/3,'totfat':70/3,'satfat':20/3,'transfat':2/3,'carbs':130/3,'protein':46/3,'fiber':25/3,'sod':2300/3,'mag':320/3,'totfolate':400/3,'potass':4700/3,'vd':600/3}
recommended_nutrient_dict_F_A = {'cal':2200/3,'sugars':90/3,'totfat':70/3,'satfat':20/3,'transfat':2/3,'carbs':130/3,'protein':46/3,'fiber':25/3,'sod':2300/3,'mag':320/3,'totfolate':400/3,'potass':4700/3,'vd':600/3}

In [10]:
# For testing, set the gender (female = 1, male = 0), set the activity level
gender = 1
activity = 0 # 0 = sedentary, 1 = moderately active, 2 = active

In [11]:
# this generates the correct dictionary depending on gender and activity
def recommended_dict(gender, activity):
    if activity == 0:
        if gender == 1:
            recommended_dict = recommended_nutrient_dict_F_S
        if gender == 0:
            recommended_dict = recommended_nutrient_dict_M_S
    if activity == 1:
        if gender == 1:
            recommended_dict = recommended_nutrient_dict_F_M
        if gender == 0:
            recommended_dict = recommended_nutrient_dict_M_M
    if activity == 2:
        if gender == 1:
            recommended_dict = recommended_nutrient_dict_F_A
        if gender == 0:
            recommended_dict = recommended_nutrient_dict_M_A
    return recommended_dict

In [12]:
def nutrientScore(nutrition_info):
    nbr_nutrients = 13
    recDict = recommended_dict(gender,activity)
    goal_list=[]
    for key in recDict:
        goal_list.append(recDict[key])
    
    nutrient_sum_list=[0]*nbr_nutrients
    for ingredient_key in nutrition_info:
        nutrition_list = list(nutrition_info[ingredient_key])
        for i in range(len(nutrition_list)):
            ingredient_nutrient_value = nutrition_list[i]
            nutrient_sum_list[i] = nutrient_sum_list[i]+ingredient_nutrient_value
    nutrient_fracs=[]
    for i in range(len(nutrient_sum_list)):
        nutrient_frac = nutrient_sum_list[i]/goal_list[i]
        nutrient_fracs.append(nutrient_frac)
    
    # Scoring
    nutrient_scores_adequacy= [None]*nbr_nutrients
    nutrient_scores_moderation = [None]*nbr_nutrients
    for i in range(len(nutrient_fracs)):
        if nutrient_fracs[i] >= 2 :       
            nutrient_scores_adequacy[i] = 4
            nutrient_scores_moderation[i] = 1
        if nutrient_fracs[i] >= 1.25 and nutrient_fracs[i] <2: 
            nutrient_scores_adequacy[i] = 4
            nutrient_scores_moderation[i] = 2
        if nutrient_fracs[i] >= 1 and nutrient_fracs[i] <1.25: 
            nutrient_scores_adequacy[i] = 4
            nutrient_scores_moderation[i] = 3
        if nutrient_fracs[i] >= .75 and nutrient_fracs[i] <1: 
            nutrient_scores_adequacy[i] = 3
            nutrient_scores_moderation[i] = 4
        if nutrient_fracs[i] >= .5 and nutrient_fracs[i] <.75: 
            nutrient_scores_adequacy[i] = 2
            nutrient_scores_moderation[i] = 4
        if nutrient_fracs[i] >= 0 and nutrient_fracs[i] <.5: 
            nutrient_scores_adequacy[i] = 1
            nutrient_scores_moderation[i] = 4
    adequacy = sum(nutrient_scores_adequacy)/(4*nbr_nutrients)
    moderation = sum(nutrient_scores_moderation)/(4*nbr_nutrients)          
            
    return adequacy, moderation    

In [26]:
def mealScore():
    scores = nutrientScore(nutrition_info_in_menu_item)
    score_avg = statistics.mean(scores)
    return score_avg